In [ ]:
# !pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 8.8 MB/s eta 0:00:00


In [54]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch




# Import basic packages for later use
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
from typing import Union
import nibabel as nib
import SimpleITK as sitk
import pickle
from ipywidgets import widgets
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from IPython.display import display, clear_output

In [29]:
# install nnunet
# !pip install nnunet

In [55]:
# check if nnunet can be imported
import nnunet

In [2]:
# unzipping brats africa data(only once)
# !unzip "/home/workstation04/GP_MMMAI/BraTSData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip" -d "/home/workstation04/GP_MMMAI/Brats_Africa"

Archive:  /home/workstation04/GP_MMMAI/BraTSData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip
   creating: /home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/
   creating: /home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/BraTS-SSA-00121-000/
  inflating: /home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/BraTS-SSA-00121-000/BraTS-SSA-00121-000-seg.nii.gz  
  inflating: /home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/BraTS-SSA-00121-000/BraTS-SSA-00121-000-t1n.nii.gz  
  inflating: /home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/BraTS-SSA-00121-000/BraTS-SSA-00121-000-t2f.nii.gz  
  inflating: /home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/BraTS-SSA-00121-000/BraTS-SSA-00121-000-t2w.nii.gz  
  inflating: /home/workstation04/GP_MMMA

In [4]:
# the number of samples in BraTS Africa dataset
# !ls "/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData" | wc -l

43


### 3 things should be changed
1. sequences naming 
2. segmentation labels
```
    . labels in BraTS2021 :{"0": "background", "1": "non-enhancing tumor", "2": "edema", "4": "enhancing tumour"}
    . labels in BraTS Africa :{"0": "background", "1": "non-enhancing tumor", "2": "edema", "3": "enhancing tumour"}
    . 3 --> 4
```
3. rotate 180 degree to the right



In [6]:
def modify_seg_file(seg_file_path):
    img = nib.load(seg_file_path)
    data = img.get_fdata()
    # Modify segmentation values
    data[data == 3] = 4

    # Save the modified segmentation file
    nib.save(nib.Nifti1Image(data, img.affine), seg_file_path)

In [7]:
# change naming of africa dataset to match BraTS2021



import os

folder_path = '/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData'  # Replace with the actual path to your folder

for subdir, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.startswith('BraTS-SSA-') and file_name.endswith('.nii.gz'):
            old_file_path = os.path.join(subdir, file_name)
            parts = file_name.split('-')
            file_prefix = parts[2]
            file_suffix = parts[4].split('.')[0]

            if 'seg' not in file_suffix:
                if 't1c' in file_suffix:
                    file_suffix = file_suffix.replace('t1c', 't1ce')
                if 't1n' in file_suffix:
                    file_suffix = file_suffix.replace('t1n', 't1')
                if 't2f' in file_suffix:
                    file_suffix = file_suffix.replace('t2f', 'flair')
                if 't2w' in file_suffix:
                    file_suffix = file_suffix.replace('t2w', 't2')
            else:
                modify_seg_file(os.path.join(folder_path,subdir ,file_name))        

            new_file_name = 'BraTS-SSA_{}_{}.nii.gz'.format(file_prefix, file_suffix)
            new_file_path = os.path.join(subdir, new_file_name)
            os.rename(old_file_path, new_file_path)


In [8]:
# changing the names of the folders from BraTS-SSA-#####-### toBraTS-SSA_#####
folder_path = '/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData'
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

for old_name in folders:
    parts = old_name.split('-')
    new_file_name = '{}-{}_{}'.format(parts[0], parts[1],parts[2])
    old_file_path = os.path.join(folder_path, old_name)
    new_file_path = os.path.join(folder_path, new_file_name)
    os.rename(old_file_path, new_file_path)



In [ ]:
import os
import nibabel as nib
import numpy as np

def rotate_nifti_file(file_path):
    # Load NIfTI data
    img = nib.load(file_path)
    data = img.get_fdata()

    # Rotate data by 180 degrees
    rotated_data = np.rot90(data, 2, axes=(0, 1))

    # Save the changes to the original file
    rotated_img = nib.Nifti1Image(rotated_data, affine=img.affine)
    nib.save(rotated_img, file_path)

def rotate_nifti_files_in_directory(directory_path):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".nii.gz"):
                file_path = os.path.join(root, file)
                rotate_nifti_file(file_path)

# Replace 'your_directory_path' with the actual path to your top-level directory
directory_path = "/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData"
rotate_nifti_files_in_directory(directory_path)


In [31]:
import shutil
import os

def copy_folder_contents(source_folder, destination_folder):

    # Ensure the destination folder exists; create it if not
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    # Get the list of files and subdirectories in the source folder
    contents = os.listdir(source_folder)

    # Iterate over each item in the source folder
    for item in contents:
        # Create the full path of the item in the source folder
        source_path = os.path.join(source_folder, item)

        # Create the full path of the item in the destination folder
        destination_path = os.path.join(destination_folder, item)

        # Copy the item to the destination folder
        if os.path.isdir(source_path):
            # If the item is a directory, recursively copy its contents
            shutil.copytree(source_path, destination_path)
        else:
            # If the item is a file, copy it
            shutil.copy2(source_path, destination_path)

# Example usage
source_folder = '/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData'
destination_folder = '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder'

copy_folder_contents(source_folder, destination_folder)

In [32]:
!ls "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder" | wc -l

43


###  Setting up nnU-Nets folder structure and environment variables
- nnUnet expects a certain folder structure and environment variables.

- Roughly they tell nnUnet:
  1. Where to look for stuff
  2. Where to put stuff

For more information about this please check: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md

In [56]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)
    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [57]:
# Set environment Variables and creating folders
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
# mount_dir = "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/nnunet_utils"
mount_dir = "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet"
path_dict = {
     "nnUNet_raw_data_base" : os.path.join(mount_dir, "nnUNet_raw"),
     "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"),
     "RESULTS_FOLDER" : os.path.join(mount_dir, "nnUNet_Results_Folder"),
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet
/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/nnUNet_raw exists.
/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/nnUNet_preprocessed exists.
/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/nnUNet_Results_Folder exists.
If No Error Occured Continue Forward. =)


## Data Preparation

In [37]:
import os
import shutil

parent_folder = "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder"

# Move all files to the parent folder
for root, dirs, files in os.walk(parent_folder):
    for file in files:
        src_path = os.path.join(root, file)
        dst_path = os.path.join(parent_folder, file)
        shutil.move(src_path, dst_path)

# Delete all subdirectories
for dir in os.listdir(parent_folder):
    dir_path = os.path.join(parent_folder, dir)
    if os.path.isdir(dir_path):
        os.rmdir(dir_path)

In [38]:
!ls "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder" | wc -l

215


In [39]:
# for prediction
# To handle the naming convension required by the model
import os
# set the path to the folder containing the files
folder_path = '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder'

# iterate over the files in the folder
for root, dirs, files in os.walk(folder_path):
    # check if the file name contains the search string
    for file in files:
      if "flair" in file:
          # replace the search string with the replace string
          new_filename = file.replace("flair", "0003")
          # rename the file
          os.rename(os.path.join(root, file), os.path.join(root, new_filename))
      if "t2" in file:
          # replace the search string with the replace string
          new_filename = file.replace("t2", "0002")
          # rename the file
          os.rename(os.path.join(root, file), os.path.join(root, new_filename))
      if "t1." in file:
          # replace the search string with the replace string
          new_filename = file.replace("t1.", "0000.")
          # rename the file
          os.rename(os.path.join(root, file), os.path.join(root, new_filename))
      if "t1ce" in file:
          # replace the search string with the replace string
          new_filename = file.replace("t1ce", "0001")
          # rename the file
          os.rename(os.path.join(root, file), os.path.join(root, new_filename))

      if  "seg" in file:
          # remove the segmentation (ground truth) if exists
          file_path = os.path.join(folder_path, file)
          # os.remove(file_path)
          # Ensure the destination folder exists; create it if not
          if not os.path.exists( '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/seg_folder'):
              os.makedirs( '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/seg_folder')
          shutil.move(file_path, '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/seg_folder')


In [17]:
filename = os.listdir(folder_path)[0]
filename = os.path.splitext(filename)[0]
filename = filename.rsplit('_', 1)[0]
print(filename)

BraTS-SSA_00074


In [58]:
if not os.path.exists( '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_folder_after12'):
              os.makedirs( '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_folder_after12')

## Inference

In [60]:
!nnUNet_predict -i "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder"  -o "/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_folder_after12" -t 501  -chk "model_latest" -m 3d_fullres --save_npz



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/nnUNet_Results_Folder/nnUNet/3d_fullres/Task501_BraTS2021/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 4 input modalities for each image
Found 43 unique case ids, here are some examples: ['BraTS-SSA_00028' 'BraTS-SSA_00074' 'BraTS-SSA_00076' 'BraTS-SSA_00037'
 'BraTS-SSA_00120' 'BraTS-SSA_00068' 'BraTS-SSA_00092' 'BraTS-SSA_00012'
 'BraTS-SSA_00116' 'BraTS-SSA_00080']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 43
number of cases that still need to be predicted: 43
emptyin

In [61]:
# this function used so that the size of softmax is consistant with the input size
# source: https://github.com/MIC-DKFZ/nnUNet/issues/1050
def save_softmax_npy_from_npz(segmentation_softmax: Union[str, np.ndarray], out_fname: str,
                                         # properties_dict: dict, order: int = 1,
                                         # region_class_order: Tuple[Tuple[int]] = None,
                                         seg_postprogess_fn: callable = None, seg_postprocess_args: tuple = None,
                                         # resampled_npz_fname: str = None,
                                         non_postprocessed_fname: str = None, # force_separate_z: bool = None,
                                         # interpolation_order_z: int = 0, verbose: bool = True
                                    ):

    seg_old_spacing = np.load(segmentation_softmax)
    seg_old_spacing = seg_old_spacing.f.softmax


    # print('segmentation_softmax[:-4] + ".pkl" :',segmentation_softmax[:-4] + ".pkl")
    file_to_read = open(segmentation_softmax[:-4] + ".pkl", "rb")
    properties_dict = pickle.load(file_to_read)
    # print('properties_dict:', properties_dict)

    shape_original_after_cropping = properties_dict.get('size_after_cropping')
    shape_original_before_cropping = properties_dict.get('original_size_of_raw_data')


    bbox = properties_dict.get('crop_bbox')
    bbox.insert(0, [0, seg_old_spacing.shape[0] + 1])
    # print("type(shape_original_before_cropping): ",type(shape_original_before_cropping))
    # print('shape_original_before_cropping: ',shape_original_before_cropping)
    shape_original_before_cropping = list(shape_original_before_cropping)
    shape_original_before_cropping.insert(0, seg_old_spacing.shape[0])
    shape_original_before_cropping = np.array(shape_original_before_cropping)
    # print('seg_old_spacing.shape: ',seg_old_spacing.shape)
    # print('shape_original_before_cropping: ',shape_original_before_cropping)
    # print(bbox)
    # return bbox

    if bbox is not None:
        seg_old_size = np.zeros(shape_original_before_cropping)
        for c in range(3):
            bbox[c][1] = np.min((bbox[c][0] + seg_old_spacing.shape[c], shape_original_before_cropping[c]))
        seg_old_size[
        bbox[0][0]:bbox[0][1],
        bbox[1][0]:bbox[1][1],
        bbox[2][0]:bbox[2][1],
        bbox[3][0]:bbox[3][1]
        ] = seg_old_spacing
    else:
        seg_old_size = seg_old_spacing

    if seg_postprogess_fn is not None:
        seg_old_size_postprocessed = seg_postprogess_fn(np.copy(seg_old_size), *seg_postprocess_args)
    else:
        seg_old_size_postprocessed = seg_old_size

        # seg_resized_itk = sitk.GetImageFromArray(np.transpose(seg_old_size_postprocessed.astype(np.float32), (3, 2, 1, 0)))
        # seg_resized_itk.SetSpacing(properties_dict['itk_spacing'])
        # seg_resized_itk.SetOrigin(properties_dict['itk_origin'])
        # seg_resized_itk.SetDirection(properties_dict['itk_direction'])
        # sitk.WriteImage(seg_resized_itk, out_fname)
        np.save(out_fname, seg_old_size_postprocessed.astype(np.float32))


    if (non_postprocessed_fname is not None) and (seg_postprogess_fn is not None):
        seg_resized_itk = sitk.GetImageFromArray(seg_old_size.astype(np.float32))
        seg_resized_itk.SetSpacing(properties_dict['itk_spacing'])
        seg_resized_itk.SetOrigin(properties_dict['itk_origin'])
        seg_resized_itk.SetDirection(properties_dict['itk_direction'])
        sitk.WriteImage(seg_resized_itk, non_postprocessed_fname)

    return seg_old_size_postprocessed.astype(np.float32)

## Get volume mask

In [62]:
def create_mask(input_nifti_path):
    # Load the NIfTI volume
    nifti_img = nib.load(input_nifti_path)
    nifti_data = nifti_img.get_fdata()
    # Create a binary mask: set to 0 where voxel value is 0 in the NIfTI volume, and set to 1 otherwise
    mask = np.where(nifti_data == 0, 0, 1)
    mask = np.transpose(mask, (2, 0, 1))

    return mask


### Uncertainty Estimation

### Estimating The Model Uncertainty Using The Entropy

In [63]:
def compute_entropy_with_mask(tensor, mask):
    sum_tensor = np.sum(tensor, axis=0, keepdims=True)
    tensor_normalized = tensor / (sum_tensor + 1e-12)
    entropy_elements = -tensor_normalized * np.log2(tensor_normalized + 1e-12)
    entropy = np.sum(entropy_elements, axis=0)

    # Multiply entropy with the mask
    entropy *= mask

    total_entropy = np.sum(entropy)

    return entropy, total_entropy

In [64]:
folder_path = '/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData'
folders_names = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders_names

['BraTS-SSA_00041',
 'BraTS-SSA_00012',
 'BraTS-SSA_00117',
 'BraTS-SSA_00049',
 'BraTS-SSA_00081',
 'BraTS-SSA_00074',
 'BraTS-SSA_00014',
 'BraTS-SSA_00026',
 'BraTS-SSA_00044',
 'BraTS-SSA_00007',
 'BraTS-SSA_00092',
 'BraTS-SSA_00080',
 'BraTS-SSA_00056',
 'BraTS-SSA_00120',
 'BraTS-SSA_00078',
 'BraTS-SSA_00113',
 'BraTS-SSA_00046',
 'BraTS-SSA_00025',
 'BraTS-SSA_00119',
 'BraTS-SSA_00047',
 'BraTS-SSA_00057',
 'BraTS-SSA_00037',
 'BraTS-SSA_00010',
 'BraTS-SSA_00050',
 'BraTS-SSA_00112',
 'BraTS-SSA_00015',
 'BraTS-SSA_00116',
 'BraTS-SSA_00055',
 'BraTS-SSA_00008',
 'BraTS-SSA_00051',
 'BraTS-SSA_00068',
 'BraTS-SSA_00093',
 'BraTS-SSA_00096',
 'BraTS-SSA_00097',
 'BraTS-SSA_00076',
 'BraTS-SSA_00110',
 'BraTS-SSA_00028',
 'BraTS-SSA_00121',
 'BraTS-SSA_00122',
 'BraTS-SSA_00115',
 'BraTS-SSA_00002',
 'BraTS-SSA_00011',
 'BraTS-SSA_00095']

In [65]:
if not os.path.exists( '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_prob_after12'):
              os.makedirs( '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_prob_after12')

In [66]:
folder_entropy_list = []
for i in range(len(folders_names)):
    # Example usage:


    filename = folders_names[i]
    # Load the .npz file
    npz_path = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_folder_after12/{filename}.npz'
    npz_data = np.load(npz_path)
    save_softmax_npy_from_npz(segmentation_softmax= f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_folder_after12/{filename}.npz', out_fname = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_prob_after12/{filename}.npy')

    probability_path = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_prob_after12/{filename}.npy'
    tensor = np.load(probability_path, allow_pickle=True)  # Shape: (num_classes, depth, height, width)

    input_nifti_path = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder/{filename}_0003.nii.gz' #make the mask from flair volume
    mask = create_mask(input_nifti_path)

    tensor = np.load(probability_path, allow_pickle=True)  # Shape: (num_classes, depth, height, width)
    entropy_data, total_entropy = compute_entropy_with_mask(tensor,mask)
    folder_entropy_list.append((filename, total_entropy))



In [49]:
sorted_folder_entropy_list = sorted(folder_entropy_list, key=lambda x: x[1], reverse=True)
# Print the sorted list
print(sorted_folder_entropy_list)

[('BraTS-SSA_00015', 201096.81), ('BraTS-SSA_00121', 183757.25), ('BraTS-SSA_00115', 161511.42), ('BraTS-SSA_00113', 151284.08), ('BraTS-SSA_00081', 147159.28), ('BraTS-SSA_00057', 141215.12), ('BraTS-SSA_00116', 132154.62), ('BraTS-SSA_00110', 126911.75), ('BraTS-SSA_00096', 102098.664), ('BraTS-SSA_00093', 100996.414), ('BraTS-SSA_00117', 99930.63), ('BraTS-SSA_00007', 95500.7), ('BraTS-SSA_00095', 92518.93), ('BraTS-SSA_00076', 87826.53), ('BraTS-SSA_00025', 84916.664), ('BraTS-SSA_00056', 82990.92), ('BraTS-SSA_00028', 79956.48), ('BraTS-SSA_00002', 79265.58), ('BraTS-SSA_00011', 79136.125), ('BraTS-SSA_00010', 75918.08), ('BraTS-SSA_00120', 71883.18), ('BraTS-SSA_00092', 68219.68), ('BraTS-SSA_00078', 54084.19), ('BraTS-SSA_00122', 46159.574), ('BraTS-SSA_00097', 42275.527), ('BraTS-SSA_00044', 38777.78), ('BraTS-SSA_00026', 37515.062), ('BraTS-SSA_00008', 36246.87), ('BraTS-SSA_00074', 35598.184), ('BraTS-SSA_00050', 34474.34), ('BraTS-SSA_00046', 34113.54), ('BraTS-SSA_00012', 3

In [67]:
sorted_folder_entropy_list = sorted(folder_entropy_list, key=lambda x: x[1], reverse=True)
# Print the sorted list
print(sorted_folder_entropy_list)

[('BraTS-SSA_00121', 163883.2), ('BraTS-SSA_00110', 154244.64), ('BraTS-SSA_00015', 147437.16), ('BraTS-SSA_00115', 143186.25), ('BraTS-SSA_00074', 141581.55), ('BraTS-SSA_00117', 138778.81), ('BraTS-SSA_00113', 121057.71), ('BraTS-SSA_00081', 114149.29), ('BraTS-SSA_00057', 104748.38), ('BraTS-SSA_00116', 103907.2), ('BraTS-SSA_00096', 103642.66), ('BraTS-SSA_00092', 95637.37), ('BraTS-SSA_00095', 85349.97), ('BraTS-SSA_00007', 82436.47), ('BraTS-SSA_00093', 79963.164), ('BraTS-SSA_00025', 79319.45), ('BraTS-SSA_00076', 78119.58), ('BraTS-SSA_00056', 68912.38), ('BraTS-SSA_00120', 65652.41), ('BraTS-SSA_00028', 65212.707), ('BraTS-SSA_00010', 63026.355), ('BraTS-SSA_00097', 61976.457), ('BraTS-SSA_00050', 60989.883), ('BraTS-SSA_00002', 60205.75), ('BraTS-SSA_00014', 56257.746), ('BraTS-SSA_00078', 53643.383), ('BraTS-SSA_00011', 53105.207), ('BraTS-SSA_00051', 52887.83), ('BraTS-SSA_00055', 51901.0), ('BraTS-SSA_00047', 48281.29), ('BraTS-SSA_00026', 48002.895), ('BraTS-SSA_00112', 4

In [68]:
# Convert the list of tuples to a NumPy array
sorted_folders_entropy_file = np.array(sorted_folder_entropy_list, dtype=[('name', '<U20'), ('value', '<f8')])

# Save the NumPy array as an .npy file
np.save('/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/sorted_folders_entropy_after12.npy', sorted_folders_entropy_file)

In [51]:
names = [item[0] for item in sorted_folder_entropy_list]
# Convert the list of names to a NumPy array
names_array = np.array(names)

# Save the NumPy array as an .npy file
np.save('/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/sorted_uncertain_folders_names_after12.npy', names_array)

In [30]:
# import pandas as pd

# # Your original list
# list_before_mask = [('BraTS-SSA_00015', 207242.23), ('BraTS-SSA_00121', 184112.97), ('BraTS-SSA_00113', 183542.42), ('BraTS-SSA_00081', 173163.1), ('BraTS-SSA_00115', 161954.08), ('BraTS-SSA_00117', 157777.67), ('BraTS-SSA_00057', 146354.56), ('BraTS-SSA_00116', 143044.42), ('BraTS-SSA_00110', 127661.23), ('BraTS-SSA_00076', 122181.664), ('BraTS-SSA_00095', 114906.29), ('BraTS-SSA_00025', 112039.03), ('BraTS-SSA_00093', 102513.53), ('BraTS-SSA_00096', 102229.95), ('BraTS-SSA_00002', 101533.98), ('BraTS-SSA_00007', 96470.8), ('BraTS-SSA_00120', 93011.78), ('BraTS-SSA_00010', 89056.836), ('BraTS-SSA_00011', 87066.64), ('BraTS-SSA_00056', 86446.24), ('BraTS-SSA_00028', 84430.234), ('BraTS-SSA_00092', 84277.09), ('BraTS-SSA_00078', 66687.34), ('BraTS-SSA_00122', 49861.54), ('BraTS-SSA_00026', 48247.45), ('BraTS-SSA_00097', 43035.723), ('BraTS-SSA_00044', 42055.88), ('BraTS-SSA_00055', 38419.586), ('BraTS-SSA_00008', 36629.312), ('BraTS-SSA_00074', 35898.836), ('BraTS-SSA_00012', 35518.105), ('BraTS-SSA_00050', 34791.42), ('BraTS-SSA_00046', 34227.35), ('BraTS-SSA_00119', 33764.484), ('BraTS-SSA_00112', 29284.982), ('BraTS-SSA_00068', 29000.904), ('BraTS-SSA_00014', 25883.176), ('BraTS-SSA_00049', 25526.51), ('BraTS-SSA_00051', 23721.645), ('BraTS-SSA_00041', 19969.082), ('BraTS-SSA_00080', 14470.328), ('BraTS-SSA_00047', 11883.364), ('BraTS-SSA_00037', 8973.3)]

# # Your list after applying the mask
# list_after_mask = [('BraTS-SSA_00015', 200986.0), ('BraTS-SSA_00121', 183570.06), ('BraTS-SSA_00115', 161242.97), ('BraTS-SSA_00113', 151170.86), ('BraTS-SSA_00081', 147099.66), ('BraTS-SSA_00057', 141199.17), ('BraTS-SSA_00116', 131698.72), ('BraTS-SSA_00110', 126894.125), ('BraTS-SSA_00096', 101851.68), ('BraTS-SSA_00093', 100822.13), ('BraTS-SSA_00117', 100445.54), ('BraTS-SSA_00007', 95307.43), ('BraTS-SSA_00095', 92638.484), ('BraTS-SSA_00076', 87833.07), ('BraTS-SSA_00025', 84988.66), ('BraTS-SSA_00056', 82886.414), ('BraTS-SSA_00028', 79911.76), ('BraTS-SSA_00002', 79276.57), ('BraTS-SSA_00011', 79192.29), ('BraTS-SSA_00010', 75851.96), ('BraTS-SSA_00120', 71894.484), ('BraTS-SSA_00092', 68250.41), ('BraTS-SSA_00078', 54100.508), ('BraTS-SSA_00122', 46046.55), ('BraTS-SSA_00097', 42240.977), ('BraTS-SSA_00044', 38759.61), ('BraTS-SSA_00026', 37251.418), ('BraTS-SSA_00008', 36309.203), ('BraTS-SSA_00074', 35625.76), ('BraTS-SSA_00050', 34625.383), ('BraTS-SSA_00046', 34098.85), ('BraTS-SSA_00012', 33130.484), ('BraTS-SSA_00119', 32684.5), ('BraTS-SSA_00055', 31845.047), ('BraTS-SSA_00112', 25646.516), ('BraTS-SSA_00051', 23366.438), ('BraTS-SSA_00049', 23146.02), ('BraTS-SSA_00014', 21010.52), ('BraTS-SSA_00068', 18588.24), ('BraTS-SSA_00041', 16012.532), ('BraTS-SSA_00080', 13869.531), ('BraTS-SSA_00037', 6153.891), ('BraTS-SSA_00047', 2695.9443)]

# # Create DataFrames
# df_before_mask = pd.DataFrame(list_before_mask, columns=['Name', 'Value_before_mask'])
# df_after_mask = pd.DataFrame(list_after_mask, columns=['Name', 'Value_after_mask'])

# # Merge DataFrames on the 'Name' column
# df_comparison = pd.merge(df_before_mask, df_after_mask, on='Name', how='inner')

# df_comparison

In [52]:
# Load the .npy file
sorted_uncertain_folders_names = np.load('/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/sorted_uncertain_folders_names_b4.npy')

# Print the loaded data
print(sorted_uncertain_folders_names)

['BraTS-SSA_00015' 'BraTS-SSA_00121' 'BraTS-SSA_00115' 'BraTS-SSA_00113'
 'BraTS-SSA_00081' 'BraTS-SSA_00057' 'BraTS-SSA_00116' 'BraTS-SSA_00110'
 'BraTS-SSA_00096' 'BraTS-SSA_00093' 'BraTS-SSA_00117' 'BraTS-SSA_00007'
 'BraTS-SSA_00095' 'BraTS-SSA_00076' 'BraTS-SSA_00025' 'BraTS-SSA_00056'
 'BraTS-SSA_00028' 'BraTS-SSA_00002' 'BraTS-SSA_00011' 'BraTS-SSA_00010'
 'BraTS-SSA_00120' 'BraTS-SSA_00092' 'BraTS-SSA_00078' 'BraTS-SSA_00122'
 'BraTS-SSA_00097' 'BraTS-SSA_00044' 'BraTS-SSA_00026' 'BraTS-SSA_00008'
 'BraTS-SSA_00074' 'BraTS-SSA_00050' 'BraTS-SSA_00046' 'BraTS-SSA_00012'
 'BraTS-SSA_00119' 'BraTS-SSA_00055' 'BraTS-SSA_00112' 'BraTS-SSA_00051'
 'BraTS-SSA_00049' 'BraTS-SSA_00014' 'BraTS-SSA_00068' 'BraTS-SSA_00041'
 'BraTS-SSA_00080' 'BraTS-SSA_00037' 'BraTS-SSA_00047']


In [53]:
import os
import shutil
import csv

# Source and destination folder paths
source_folder = '/home/workstation04/GP_MMMAI/Brats_Africa/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData'
destination_folder = '/home/workstation04/GP_MMMAI/All_Data'


# Create destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Copy the first 10 folders
mapping = []
for i, folder_name in enumerate(sorted_uncertain_folders_names[:10], start=1667):
    source_path = os.path.join(source_folder, folder_name)
    new_folder_name = f'BraTS2021_{i:05d}'
    destination_path = os.path.join(destination_folder, new_folder_name)
    
    # Copy the folder
    shutil.copytree(source_path, destination_path)
    
    # Rename NIfTI files inside the copied folder
    for file_name in os.listdir(destination_path):
        if file_name.endswith('.nii.gz'):
            old_file_path = os.path.join(destination_path, file_name)
            new_file_name = f'BraTS2021_{i:05d}{file_name[len(folder_name):]}'
            new_file_path = os.path.join(destination_path, new_file_name)
            
            # Rename the file
            os.rename(old_file_path, new_file_path)
    
    # Add mapping to the list
    mapping.append((folder_name, new_folder_name))

# Save the mapping to CSV
csv_file_path = '/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/map1.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Previous Name', 'New Name'])
    csv_writer.writerows(mapping)

print(f'Folders copied, files renamed, and mapping saved to {csv_file_path}')


Folders copied, files renamed, and mapping saved to /home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/map1.csv


### Interactive Visualization

In [11]:
from ipywidgets import widgets
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from IPython.display import display, clear_output

#     labels_dict = {"0": "background", "1": "edema", "2": "non-enhancing tumor", "3": "enhancing tumour"}
# africa 1 et 2 necrosic 3 edema
def plot_single_slice(file_name, slice_index, input_data, prediction_data, entropy_data):

    labels = ['background', 'edema', 'necrosis', 'enhancing tumor']
    colors = ['black',  'white',  'red', 'green']

    cmap = ListedColormap(colors) # Define the colormap

    fig, ax = plt.subplots(1, 4, figsize=(15, 7))
    # Assuming you have defined alpha_channel as mentioned before
    alpha_channel = np.where(input_data[:, :, slice_index] == 0, 0, 0.7)
 

    ax[3].imshow(input_data[:, :, slice_index], cmap='gray')  
    ax[3].imshow(entropy_data[:, :, slice_index], cmap='hot', alpha=alpha_channel)  
    ax[3].set_title('Uncertainty on top of Input')


    ax[2].imshow(entropy_data[ :, :,slice_index], cmap='inferno')  # Using a colormap better suited for entropy
    ax[2].set_title(f'{file_name}')

    ax[1].imshow(prediction_data[ :, :,slice_index], cmap=cmap)
    ax[1].set_title('Segmentation')

    ax[0].imshow(input_data[ :, :,slice_index], cmap='gray')
    ax[0].set_title('Input')

    # Create a legend
    patches = [mpatches.Patch(color=colors[i], label=label) for i, label in enumerate(labels)]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    plt.suptitle(f'{file_name} - Slice {slice_index}', y=0.98)  # moving the title up slightly

    plt.show()

def plot_single_image(file_name, input_data, prediction_data, entropy_data):
    # Assuming that entropy_tensor is a 3D numpy array with shape (256, 256, 256)
    # or that load_image_data function is capable of reading it into this shape

    min_index = 0
    max_index = entropy_data.shape[2] - 1  # Changed to 0, assuming that entropy_tensor is 3D with shape (256, 256, 256)

    # Clear the output before displaying the new widget
    clear_output(wait=True)
    # Create interactive plot
    widget_instance = widgets.interact(
        plot_single_slice,
        file_name=widgets.fixed(file_name),
        slice_index=widgets.IntSlider(min=min_index, max=max_index, step=1, value=max_index // 2),
        entropy_data=widgets.fixed(entropy_data),  # Passed entropy_tensor here
        prediction_data=widgets.fixed(prediction_data),
        input_data = widgets.fixed(input_data),
        )

    display(widget_instance)




In [13]:
filename = 'BraTS-SSA_00015'
# Load the probability data
probability_path = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/output_probability/{filename}.npy'
tensor = np.load(probability_path, allow_pickle=True)  # Shape: (num_classes, depth, height, width)
input_nifti_path = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder/{filename}_0003.nii.gz' #make the mask from flair volume
mask = create_mask(input_nifti_path)
tensor = np.load(probability_path, allow_pickle=True)  # Shape: (num_classes, depth, height, width)
entropy_data, total_entropy = compute_entropy_with_mask(tensor,mask)
entropy_data = np.transpose(entropy_data, (1, 2,0 ))
# Load the input NIfTI file
input_path =f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/input_folder/{filename}_0003.nii.gz'
input_data = nib.load(input_path).get_fdata()  # Shape: (height, width, depth)
input_data = np.transpose(input_data, (1, 0, 2))


# Load the Prediction NIfTI file
prediction_path = f'/home/workstation04/GP_MMMAI/Notebooks/Vanilla_nnUNet/seg_folder/{filename}_seg.nii.gz'
prediction_data = nib.load(prediction_path).get_fdata()  # Shape: (height, width, depth)
prediction_data = np.transpose(prediction_data, (1, 0, 2))


In [14]:
# Test the function
plot_single_image('Entropy Image', input_data, prediction_data, entropy_data)

interactive(children=(IntSlider(value=77, description='slice_index', max=154), Output()), _dom_classes=('widge…

<function __main__.plot_single_slice(file_name, slice_index, input_data, prediction_data, entropy_data)>